#### 111

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] ="7"

In [95]:
from networks.utils.Attention import *

In [11]:
import torch
ckpt_path = "checkpoints/WeathBench32x64MultibranchRandom/LgNet_MultiBranch-DecodeEncodeLgNet/ts_2_pretrain-2D706N_per_1_step/03_04_16_35_38553-seed_73001/pretrain_latest.pt"
ckpt = torch.load(ckpt_path)

In [14]:
# lets pick out the 24 hour weight train
new_ckpt_path = {}
for key, val in ckpt['model'].items():
    if "net.layers.2" in key : continue
    if "net.final" in key : continue 
    if "new_branch" in key:
        if "new_branch.2." in key:
            key = key.replace("new_branch.2.0.","backbone.net.layers.2.")
            key = key.replace("new_branch.2.1.","backbone.net.final.")
            new_ckpt_path[key] = val
        continue
    new_ckpt_path[key] = val

In [16]:
target_path = "checkpoints/WeathBench32x64/CK_LgNet/ts_2_pretrain-2D706N_per_24_step/branch_from_multibranch/pretrain_latest.pt"
new_ckpt={}
new_ckpt['model'] = new_ckpt_path
torch.save(new_ckpt,target_path)

In [15]:
for k in new_ckpt_path.keys():
    print(f"{k:10s}")

backbone.net.patch_embed.proj.weight
backbone.net.patch_embed.proj.bias
backbone.net.layers.0.blocks.0.norm.weight
backbone.net.layers.0.blocks.0.norm.bias
backbone.net.layers.0.blocks.0.attn.qkv.weight
backbone.net.layers.0.blocks.0.attn.qkv.bias
backbone.net.layers.0.blocks.0.attn.proj.weight
backbone.net.layers.0.blocks.0.attn.proj.bias
backbone.net.layers.0.blocks.0.norm2.weight
backbone.net.layers.0.blocks.0.norm2.bias
backbone.net.layers.0.blocks.0.mlp.fc1.weight
backbone.net.layers.0.blocks.0.mlp.fc1.bias
backbone.net.layers.0.blocks.0.mlp.fc2.weight
backbone.net.layers.0.blocks.0.mlp.fc2.bias
backbone.net.layers.0.blocks.1.norm.weight
backbone.net.layers.0.blocks.1.norm.bias
backbone.net.layers.0.blocks.1.attn.qkv.weight
backbone.net.layers.0.blocks.1.attn.qkv.bias
backbone.net.layers.0.blocks.1.attn.proj.weight
backbone.net.layers.0.blocks.1.attn.proj.bias
backbone.net.layers.0.blocks.1.norm2.weight
backbone.net.layers.0.blocks.1.norm2.bias
backbone.net.layers.0.blocks.1.mlp.f

In [2]:
from networks.LGNet import LGNet

In [5]:
model = LGNet(img_size=(32,64),patch_size= [1, 1],in_chans= 71,out_chans= 138,embed_dim=1152//2,
      window_size= (4,8),depths= [4, 4, 4],num_heads= [6, 6, 6],Weather_T=1)

In [8]:

with torch.no_grad():
    x = torch.randn(1,71,32,64)
    B = x.shape[0]
    x, T, H, W = model.net.patch_embed(x)  # x:[B, H*W, C]
    x = x + model.net.pos_embed
    x = model.net.pos_drop(x)
    if len(model.net.window_size) == 3:
        x = x.view(B, T, H, W, -1)
    elif len(model.net.window_size) == 2:
        x = x.view(B, H, W, -1)

    for layer in model.net.layers:
        x= layer(x);print(x.shape)

    x = model.net.final(x)

torch.Size([1, 32, 64, 576])
torch.Size([1, 32, 64, 576])
torch.Size([1, 32, 64, 576])


In [3]:
import torch
import torch.nn as nn

In [7]:
class PatchEmbed(nn.Module):
    """
    2D Image to Patch Embedding
    """
    def __init__(self, patch_size=[1, 1, 1], in_c=3, embed_dim=96, norm_layer=None):
        super().__init__()
        self.patch_size = patch_size
        self.in_chans = in_c
        self.embed_dim = embed_dim
        if len(patch_size) == 2:
            self.proj = nn.Conv2d(in_c, embed_dim, kernel_size=patch_size, stride=patch_size)
        elif len(patch_size) == 3:
            self.proj = nn.Conv3d(in_c, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        if len(self.patch_size) == 3:
            _, _, T, H, W = x.shape
        elif len(self.patch_size) == 2:
            _, _, H, W = x.shape

        # 下采样patch_size倍
        x = self.proj(x)
        # _, _, T, H, W = x.shape
        # flatten: [B, C, H, W] -> [B, C, HW]
        # transpose: [B, C, HW] -> [B, HW, C]
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)
        if len(self.patch_size) == 3:
            return x, T//self.patch_size[-3], H//self.patch_size[-2], W//self.patch_size[-1]
        elif len(self.patch_size) == 2:
            return x, 1, H//self.patch_size[-2], W//self.patch_size[-1]

In [13]:
class PatchEmbed_1Patch(nn.Module):
    """
    2D Image to Patch Embedding
    """
    def __init__(self, patch_size=[1, 1], in_c=3, embed_dim=96, norm_layer=None):
        super().__init__()
        self.patch_size = patch_size
        self.in_chans = in_c
        self.embed_dim = embed_dim
        assert tuple(patch_size)==(1,1)
        self.proj = nn.Linear(in_c, embed_dim)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        _, _, H, W = x.shape

        # 下采样patch_size倍
        x = self.proj(x.permute(0,2,3,1)).permute(0,3,1,2)
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)
        return x, 1, H//self.patch_size[-2], W//self.patch_size[-1]

In [ ]:
torch.split()

In [20]:
up_branch,dw_branch = torch.split(b[0],512,-1)

In [33]:

base2  = 1000
theta2 = torch.pow(base2, -torch.arange(512)/512)

In [102]:
class Rotaty2DEmbedding(nn.Module):
    def __init__(self, embed_dim=512, w=32, h=64, base_w=1000, base_h=1000):
        super().__init__()
        theta_h = torch.pow(base1, -torch.arange(embed_dim)/embed_dim)
        mtheta_h= torch.einsum("i,j->ij",torch.arange(w),theta_h)
        cos_m_theta_h = torch.cos(mtheta_h).unsqueeze(1).repeat(1,h,1).reshape(1,w*h,embed_dim)
        sin_m_theta_h = torch.sin(mtheta_h).unsqueeze(1).repeat(1,h,1).reshape(1,w*h,embed_dim)
        theta_w = torch.pow(base1, -torch.arange(embed_dim)/embed_dim)
        mtheta_w= torch.einsum("i,j->ij",torch.arange(h),theta_w)
        cos_m_theta_w = torch.cos(mtheta_w).unsqueeze(0).repeat(w,1,1).reshape(1,w*h,embed_dim)
        sin_m_theta_w = torch.sin(mtheta_w).unsqueeze(0).repeat(w,1,1).reshape(1,w*h,embed_dim)
        self.embed_dim = embed_dim
        self.register_buffer("cos_m_theta_h",cos_m_theta_h)
        self.register_buffer("sin_m_theta_h",sin_m_theta_h)
        self.register_buffer("cos_m_theta_w",cos_m_theta_w)
        self.register_buffer("sin_m_theta_w",sin_m_theta_w)
    def forward(self,x):
        assert x.shape[-1] == 2*self.embed_dim
        up_branch, dw_branch = torch.split(x,self.embed_dim,-1)
        L1 = up_branch*self.cos_m_theta_h - dw_branch*self.sin_m_theta_h
        L2 = up_branch*self.sin_m_theta_h + dw_branch*self.cos_m_theta_h
        L3 = up_branch*self.cos_m_theta_w - dw_branch*self.sin_m_theta_w
        L4 = up_branch*self.sin_m_theta_w + dw_branch*self.cos_m_theta_w
        return torch.stack([L1,L2,L3,L4],-1).flatten(-2,-1)

In [103]:
layer = Rotaty2DEmbedding()

In [104]:
a  = torch.randn(1, 2048, 1024)

In [105]:
layer(a).shape

torch.Size([1, 2048, 2048])

In [15]:
a = torch.randn(1,71,32,64).cuda()
model1 = PatchEmbed((1,1),in_c=71, embed_dim=1024)
model1 = model1.cuda()
model2 = PatchEmbed_1Patch((1,1),in_c=71, embed_dim=1024)
model2 = model2.cuda()

In [16]:
%%timeit
with torch.no_grad():
    b = model1(a)

80.2 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
%%timeit
with torch.no_grad():
    b = model2(a)

55.3 µs ± 306 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
model = LGNet(img_size=(64,128),patch_size= [1, 1],in_chans= 71,out_chans= 138,embed_dim=1152,
      window_size= (4,8),depths= [4, 4, 4],num_heads= [6, 6, 6],Weather_T=1)

In [32]:
import timm, tome

# Load a pretrained model, can be any vit / deit model.
model2 = timm.create_model("vit_base_patch16_224", pretrained=False)

In [14]:
tome.patch.timm(model)

In [5]:
from tome.patch.timm import *

In [6]:
def apply_patch(model, trace_source: bool = False, prop_attn: bool = True):
    """
    Applies ToMe to this transformer. Afterward, set r using model.r.
    If you want to know the source of each token (e.g., for visualization), set trace_source = true.
    The sources will be available at model._tome_info["source"] afterward.
    For proportional attention, set prop_attn to True. This is only necessary when evaluating models off
    the shelf. For trianing and for evaluating MAE models off the self set this to be False.
    """
    ToMeLgNet = make_tome_class(model.__class__)

    model.__class__ = ToMeLgNet
    model.r = 0
    model._tome_info = {
        "r": model.r,
        "size": None,
        "source": None,
        "trace_source": trace_source,
        "prop_attn": prop_attn,
        "class_token": None,
        "distill_token": False,
    }

    for layer in model.net.layers:
        for module in layer.blocks.modules():
            if isinstance(module, Windowattn_block):
                module.__class__ = ToMeBlock
                module._tome_info = model._tome_info
            elif isinstance(module, SD_attn):
                module.__class__ = ToMeAttention

In [16]:
ToMeLgNet = make_tome_class(model.__class__)

model.__class__ = ToMeLgNet
model.r = 0
model._tome_info = {
    "r": model.r,
    "size": None,
    "source": None,
    "trace_source": False,
    "prop_attn": True,
    "class_token": None,
    "distill_token": False,
}
for layer in model.net.layers:
    for module in layer.blocks.modules():
        if isinstance(module, Windowattn_block):
            module.__class__ = ToMeBlock
            module._tome_info = model._tome_info
        elif isinstance(module, SD_attn):
            module.__class__ = ToMeAttention

In [19]:
block = model.net.layers[0].blocks[0]

SD_attn(
  (rope_quad): rope2()
  (qkv): Linear(in_features=1152, out_features=3456, bias=True)
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=1152, out_features=1152, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
  (softmax): Softmax(dim=-1)
  (position_enc): rope2()
)

In [ ]:
from typing import Tuple

import torch
from networks.LGNet import *
from tome.merge import bipartite_soft_matching, merge_source, merge_wavg
from tome.utils import parse_r


class ToMeBlock(Block):
    """
    Modifications:
     - Apply ToMe between the attention and mlp blocks
     - Compute and propogate token size and potentially the token sources.
    """

    def _drop_path1(self, x):
        return self.drop_path1(x) if hasattr(self, "drop_path1") else self.drop_path(x)

    def _drop_path2(self, x):
        return self.drop_path2(x) if hasattr(self, "drop_path2") else self.drop_path(x)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Note: this is copied from timm.models.vision_transformer.Block with modifications.
        attn_size = self._tome_info["size"] if self._tome_info["prop_attn"] else None
        x_attn, metric = self.attn(self.norm1(x), attn_size)
        x = x + self._drop_path1(x_attn)

        r = self._tome_info["r"].pop(0)
        if r > 0:
            # Apply ToMe here
            merge, _ = bipartite_soft_matching(
                metric,
                r,
                self._tome_info["class_token"],
                self._tome_info["distill_token"],
            )
            if self._tome_info["trace_source"]:
                self._tome_info["source"] = merge_source(
                    merge, x, self._tome_info["source"]
                )
            x, self._tome_info["size"] = merge_wavg(merge, x, self._tome_info["size"])

        x = x + self._drop_path2(self.mlp(self.norm2(x)))
        return x

In [22]:
block.attn

SD_attn(
  (rope_quad): rope2()
  (qkv): Linear(in_features=1152, out_features=3456, bias=True)
  (attn_drop): Dropout(p=0.0, inplace=False)
  (proj): Linear(in_features=1152, out_features=1152, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
  (softmax): Softmax(dim=-1)
  (position_enc): rope2()
)

In [ ]:
class ToMeAttention(Attention):
    """
    Modifications:
     - Apply proportional attention
     - Return the mean of k over heads from attention
    """

    def forward(
        self, x: torch.Tensor, size: torch.Tensor = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        # Note: this is copied from timm.models.vision_transformer.Attention with modifications.
        B, N, C = x.shape
        qkv = (
            self.qkv(x)
            .reshape(B, N, 3, self.num_heads, C // self.num_heads)
            .permute(2, 0, 3, 1, 4)
        )
        q, k, v = (
            qkv[0],
            qkv[1],
            qkv[2],
        )  # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale

        # Apply proportional attention
        if size is not None:
            attn = attn + size.log()[:, None, None, :, 0]

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)

        # Return k as well here
        return x, k.mean(1)


def make_tome_class(transformer_class):
    class ToMeVisionTransformer(transformer_class):
        """
        Modifications:
        - Initialize r, token size, and token sources.
        """

        def forward(self, *args, **kwdargs) -> torch.Tensor:
            self._tome_info["r"] = parse_r(len(self.blocks), self.r)
            self._tome_info["size"] = None
            self._tome_info["source"] = None

            return super().forward(*args, **kwdargs)

    return ToMeVisionTransformer

In [ ]:
a = torch.randn(1,71,64,128)

In [5]:
weight = torch.load("../wpredict-master/checkpoint/lgnet_finetune/world_size8-swinvrnn64x128_lgnet_possloss_finetune_norandom/checkpoint_best.pth", map_location='cpu')
good_weight = dict((key.replace("module.",""),val) for key,val in weight['model']['lgnet'].items())

FileNotFoundError: [Errno 2] No such file or directory: '../wpredict-master/checkpoint/lgnet_finetune/world_size8-swinvrnn64x128_lgnet_possloss_finetune_norandom/checkpoint_best.pth'

In [18]:
model.load_state_dict(good_weight,strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['max_logvar', 'min_logvar'])

In [19]:
model = model.cuda()

In [11]:
new_model = convert_to_int8(model)

In [20]:
a = torch.randn(1,71,64,128).cuda()

In [21]:
b = model(a)

In [23]:
convert_to_int8(model)
model = model.cuda()
c = model(a)


/nvme/zhangtianning/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)


In [24]:
torch.dist(b,c)

tensor(4017.6440, device='cuda:0', grad_fn=<DistBackward0>)

In [5]:
weightpath = "checkpoints/WeathBench7066/GraphCastDGLSym/ts_2_pretrain-2D706N_per_1_step/02_12_16_39_55711-seed_73001"

In [8]:
from model.GraphCast import *

In [9]:
from train.pretrain import get_args
args=get_args(os.path.join(weightpath,"config.json"))

In [10]:
import torch

In [11]:
weigth = torch.load(os.path.join(weightpath,"backbone.best.pt"))['model']

In [12]:
model1 = GraphCastDGLSym(img_size=(64,128), embed_dim=512,graphflag='mesh6')


            This is ===> GraphCast Model(DGL) <===
            Information: 
                total mesh node:10242 total unique mesh edge:40950*2=81900 
                total grid node 8192+2 = 8194 but activated grid 7790 
                from activated grid to mesh, create 4*10242 - 6 = 40962 edges. (north and south pole repeat 4 times) 
                there are 404 unactivated grid node
                when mapping node to grid, 
                from node to activated grid, there are 40030 edges
                from node to unactivated grid, there are 3232 edges
                thus, totally have 43262 edge. 
                #notice some grid only have 1-2 linked node but some grid may have 30 lined node
            


In [13]:
for p in model1.grid_rect_embedding_layer.parameters():
    print(p.requires_grad)

True
True


In [58]:
model1   = model1.cuda()
optimizer= torch.optim.SGD(model1.parameters(), lr=0.1)
a= torch.randn(1,70,64,128).cuda()
b= torch.randn(1,70,64,128).cuda()


In [61]:
%%timeit
optimizer.zero_grad()
loss = torch.nn.MSELoss()(model1(a),b)
loss.backward()
optimizer.step()

147 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
model1.northsouthembbed.requires_grad         = False
model1.mesh_node_embedding.requires_grad      = False
model1.mesh_mesh_bond_embedding.requires_grad = False
model1.grid_mesh_bond_embedding.requires_grad = False

In [63]:
%%timeit
optimizer.zero_grad()
loss = torch.nn.MSELoss()(model1(a),b)
loss.backward()
optimizer.step()

142 ms ± 54.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [75]:
model2 = LoRAGraphCastDGLSym(img_size=(64,128), embed_dim=512,graphflag='mesh6')


            This is ===> GraphCast Model(DGL) <===
            Information: 
                total mesh node:10242 total unique mesh edge:40950*2=81900 
                total grid node 8192+2 = 8194 but activated grid 7790 
                from activated grid to mesh, create 4*10242 - 6 = 40962 edges. (north and south pole repeat 4 times) 
                there are 404 unactivated grid node
                when mapping node to grid, 
                from node to activated grid, there are 40030 edges
                from node to unactivated grid, there are 3232 edges
                thus, totally have 43262 edge. 
                #notice some grid only have 1-2 linked node but some grid may have 30 lined node
            


In [76]:
model2   = model2.cuda()
optimizer= torch.optim.SGD(model2.parameters(), lr=0.1)
a= torch.randn(1,70,64,128).cuda()
b= torch.randn(1,70,64,128).cuda()

In [77]:
%%timeit
optimizer.zero_grad()
loss = torch.nn.MSELoss()(model2(a),b)
loss.backward()
optimizer.step()

148 ms ± 9.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
class LoRAGraphCastDGLSym(GraphCastDGLBase): 
    def __init__(self, img_size=(32,64),  in_chans=70, out_chans=70, depth=6, embed_dim=128, graphflag='mesh5', nonlinear='swish', **kargs):
        super().__init__()

        g = self.build_dgl(graphflag,img_size)

        #### build block ####
        edge_flag = ('mesh', 'M2M', 'mesh')
        self.grid2mesh = LoRANode2Edge2NodeBlockDGL('grid','G2M','mesh',embed_dim=embed_dim,do_source_update=True)
        self.mesh2mesh = nn.ModuleList()
        for i in range(depth):self.mesh2mesh.append(LoRANode2Edge2NodeBlockDGLSymmetry('mesh','M2M','mesh',embed_dim=embed_dim))        
        self.mesh2grid = LoRANode2Edge2NodeBlockDGL('mesh','M2G','grid',embed_dim=embed_dim)

        self.grid_rect_embedding_layer = nn.Linear(in_chans,embed_dim)
        self.projection                = nn.Linear(embed_dim,out_chans)
        
        self.register_buffer('northsouthembbed',        torch.randn(2,embed_dim).requires_grad_(False))
        self.register_buffer('mesh_node_embedding',     torch.randn(g.num_nodes('mesh'),1, embed_dim).requires_grad_(False))
        self.register_buffer('grid_mesh_bond_embedding',torch.randn(g.num_edges('G2M'),1, embed_dim).requires_grad_(False))
        self.register_buffer('mesh_grid_bond_template',torch.randn(g.num_edges('M2G'),1, embed_dim).requires_grad_(False))
        self.g = g
        self.embed_dim = embed_dim
        M2Mweightorder1= g.edge_ids(self.M2M_edgeid2pair[:,0],self.M2M_edgeid2pair[:,1],etype='M2M')
        M2Mweightorder2= g.edge_ids(self.M2M_edgeid2pair[:,1],self.M2M_edgeid2pair[:,0],etype='M2M')
        mesh_mesh_bond_embedding = torch.randn(g.num_edges('M2M'),1, embed_dim)
        mesh_mesh_bond_embedding[M2Mweightorder2] = mesh_mesh_bond_embedding[M2Mweightorder2]    
        self.register_buffer('mesh_mesh_bond_embedding',mesh_mesh_bond_embedding.requires_grad_(False))
        #self.mesh_mesh_bond_embedding  = nn.Parameter(mesh_mesh_bond_embedding)


In [16]:
model1 = GraphCastDGLSym(img_size=(32,64), embed_dim=512)
model1.load_state_dict(weigth)


            This is ===> GraphCast Model(DGL) <===
            Information: 
                total mesh node: 2562 total unique mesh edge:10230*2=20460 
                total grid node 2048+2 = 2050 but activated grid 1928 
                from activated grid to mesh, create 4*2562 - 6 = 10242 edges. (north and south pole repeat 4 times) 
                there are 122 unactivated grid node
                when mapping node to grid, 
                from node to activated grid, there are 10032 edges
                from node to unactivated grid, there are 976 edges
                thus, totally have 11008 edge. 
                #notice some grid only have 1-2 linked node but some grid may have 30 lined node
            


<All keys matched successfully>

In [ ]:
for p in model1.grid_rect_embedding_layer.parameters():
    print(p.requires_grad)

In [54]:
from mltool.universal_model_util import get_model_para_detail

In [33]:
b  = torch.randn(1,2,512)

In [45]:
torch.dist(model2.grid2mesh.STE2E_E2E.main.weight,
           model1.grid2mesh.STE2E_E2E
              )

tensor(0., grad_fn=<DistBackward0>)

In [47]:
out1 = b@model1.grid2mesh.STE2E_E2E
out2 = model2.grid2mesh.STE2E_E2E(b)
torch.dist(out1,out2)

tensor(8.7202e-06, grad_fn=<DistBackward0>)

In [46]:
model2 = LoRAGraphCastDGLSym(img_size=(32,64), embed_dim=512)
new_weight = {}
for key, w in weigth.items():
    skip=False
    for activate_key in ['STE2E_S2E','STE2E_T2E','STE2E_E2E',
                         'ET2T_E2T','ET2T_T2T','S2S']:
        if activate_key in key:
            key += ".main.weight"
            new_weight[key] = w.transpose(1,0)
            skip = True
            continue
            
    if skip:continue
    new_weight[key] = w
model2.load_state_dict(new_weight)


            This is ===> GraphCast Model(DGL) <===
            Information: 
                total mesh node: 2562 total unique mesh edge:10230*2=20460 
                total grid node 2048+2 = 2050 but activated grid 1928 
                from activated grid to mesh, create 4*2562 - 6 = 10242 edges. (north and south pole repeat 4 times) 
                there are 122 unactivated grid node
                when mapping node to grid, 
                from node to activated grid, there are 10032 edges
                from node to unactivated grid, there are 976 edges
                thus, totally have 11008 edge. 
                #notice some grid only have 1-2 linked node but some grid may have 30 lined node
            


<All keys matched successfully>

In [13]:
a = torch.randn(1,70,32,64)

In [20]:
from einops import rearrange

In [30]:
with torch.no_grad():
    _input = a.clone()
    B, P, W, H = _input.shape
    device = next(model1.parameters()).device
    if model1.device is None:
        model1.device = device
        model1.g = model1.g.to(device)
    # (B,P,W,H) -> (B,W*H,P)
    feature_along_latlot = model1.grid_rect_embedding_layer(
        rearrange(_input, "B P W H -> (W H) B P"))
    # (L,B,D)
    grid_rect_embedding1 = feature_along_latlot[model1.G2M_grid2LaLotudePos]
    grid_rect_embedding1 = torch.cat([rearrange(model1.northsouthembbed.repeat(B, 1, 1), "B L D -> L B D"),
                                        grid_rect_embedding1])  # --> (L+2, B, D)
    L = len(grid_rect_embedding1)
    g = model1.g
    g.nodes['grid'].data['feat'] = torch.nn.functional.pad(
        grid_rect_embedding1, (0, 0, 0, 0, 0, model1.unactivated_grid))
    g.nodes['mesh'].data['feat'] = model1.mesh_node_embedding
    g.edges['G2M'].data['feat'] = model1.grid_mesh_bond_embedding
    g.edges['M2M'].data['feat'] = model1.mesh_mesh_bond_embedding
    g.edges['G2M'].data['coef'] = model1.G2M_edge_coef.to(device)
    g.edges['M2G'].data['coef'] = model1.M2G_edge_coef.to(device)
    # checknan(g,'initial');
    g = model1.grid2mesh(g)
#     # checknan(g,'grid2mesh');
#     for layer_idx, mesh2mesh in enumerate(model1.mesh2mesh):
#         g = mesh2mesh(g)  # checknan(g,f"mesh2mesh_{layer_idx}")
#     g.edges['M2G'].data['feat'] = torch.nn.functional.pad(
#         g.edges['G2M'].data['feat'][model1.reorder_edge_id_of_M2G_from_G2M], (0, 0, 0, 0, 0, model1.num_unactivated_edge))
#     # luckly, the model1.reorder_edge_id_in_M2G is just np.arange(....) thus, use padding rather than create one can fast 50%
#     # model1.mesh_grid_bond_template = torch.zeros(g.num_edges('M2G'),B,model1.embed_dim).to(g.edges['G2M'].data['feat'].device)
#     # model1.mesh_grid_bond_template[model1.reorder_edge_id_in_M2G] = g.edges['G2M'].data['feat'][model1.reorder_edge_id_of_M2G_from_G2M]
#     # g.edges['M2G'].data['feat'] = model1.mesh_grid_bond_template
#     # g.nodes['grid'].data['feat'][L:] = 0 <--- only to align with GraphCastFast
#     g = model1.mesh2grid(g)  # checknan(g,'mesh2grid')
#     # (64,128,B,embed_dim)
#     out = g.nodes['grid'].data['feat'][model1.M2G_LaLotudePos2grid]

In [32]:
torch.dist(model1.g.nodes['grid'].data['feat'],
           model2.g.nodes['grid'].data['feat'])

tensor(1270.1317)

In [31]:
with torch.no_grad():
    _input = a.clone()
    B, P, W, H = _input.shape
    device = next(model2.parameters()).device
    if model2.device is None:
        model2.device = device
        model2.g = model2.g.to(device)
    # (B,P,W,H) -> (B,W*H,P)
    feature_along_latlot = model2.grid_rect_embedding_layer(
        rearrange(_input, "B P W H -> (W H) B P"))
    # (L,B,D)
    grid_rect_embedding2 = feature_along_latlot[model2.G2M_grid2LaLotudePos]
    grid_rect_embedding2 = torch.cat([rearrange(model2.northsouthembbed.repeat(B, 1, 1), "B L D -> L B D"),
                                        grid_rect_embedding2])  # --> (L+2, B, D)
    L = len(grid_rect_embedding2)
    g = model2.g
    g.nodes['grid'].data['feat'] = torch.nn.functional.pad(
        grid_rect_embedding2, (0, 0, 0, 0, 0, model2.unactivated_grid))
    g.nodes['mesh'].data['feat'] = model2.mesh_node_embedding
    g.edges['G2M'].data['feat'] = model2.grid_mesh_bond_embedding
    g.edges['M2M'].data['feat'] = model2.mesh_mesh_bond_embedding
    g.edges['G2M'].data['coef'] = model2.G2M_edge_coef.to(device)
    g.edges['M2G'].data['coef'] = model2.M2G_edge_coef.to(device)
    # checknan(g,'initial');
    g = model2.grid2mesh(g)
#     # checknan(g,'grid2mesh');
#     for layer_idx, mesh2mesh in enumerate(model2.mesh2mesh):
#         g = mesh2mesh(g)  # checknan(g,f"mesh2mesh_{layer_idx}")
#     g.edges['M2G'].data['feat'] = torch.nn.functional.pad(
#         g.edges['G2M'].data['feat'][model2.reorder_edge_id_of_M2G_from_G2M], (0, 0, 0, 0, 0, model2.num_unactivated_edge))
#     # luckly, the model2.reorder_edge_id_in_M2G is just np.arange(....) thus, use padding rather than create one can fast 50%
#     # model2.mesh_grid_bond_template = torch.zeros(g.num_edges('M2G'),B,model2.embed_dim).to(g.edges['G2M'].data['feat'].device)
#     # model2.mesh_grid_bond_template[model2.reorder_edge_id_in_M2G] = g.edges['G2M'].data['feat'][model2.reorder_edge_id_of_M2G_from_G2M]
#     # g.edges['M2G'].data['feat'] = model2.mesh_grid_bond_template
#     # g.nodes['grid'].data['feat'][L:] = 0 <--- only to align with GraphCastFast
#     g = model2.mesh2grid(g)  # checknan(g,'mesh2grid')
#     # (64,128,B,embed_dim)
#     out = g.nodes['grid'].data['feat'][model2.M2G_LaLotudePos2grid]

In [26]:
from mltool.universal_model_util import get_model_para_detail

In [10]:
model1 = model1.cuda()

In [16]:
a = torch.randn(1,70,32,64).cuda()
b = torch.randn(1,70,32,64).cuda()

In [18]:
%%timeit
loss = torch.nn.MSELoss()(b,model1(a))
loss.backward()

43 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
model1.northsouthembbed.requires_grad = False
model1.mesh_node_embedding.requires_grad = False
model1.mesh_mesh_bond_embedding.requires_grad = False
model1.grid_mesh_bond_embedding.requires_grad = False

In [20]:
%%timeit
loss = torch.nn.MSELoss()(b,model1(a))
loss.backward()

44.4 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


NameError: name 'model1' is not defined

In [3]:
import torch

In [ ]:
local_mask = torch.ones(seq_len, seq_len, dtype=torch.bool)

In [7]:
torch.nn.functional.conv2d(torch.ones(1,1,16,16),torch.ones(1,1,3,3))

tensor([[[[9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
          [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.]]]])

In [19]:
from einops import rearrange

In [ ]:

from typing import Optional
import torch
from torch import nn

class AFTLocalConv2D(nn.Module):
    """
    ### AFT Local Operation
    $$Y_t = \sigma(Q_t) \odot
     \frac{\sum_{t'=1}^T \exp(K_{t'} + w_{t,t'}) \odot V_{t'}}
     {\sum_{t'=1}^T \exp(K_{t'} + w_{t,t'})}$$
    where,
    \begin{align}
    w'_{t,t'} =
    \begin{cases}
    w_{t,t'},  & {\text{for } \lvert t-t' \rvert \lt s} \\
    0, & \text{otherwise}
    \end{cases}
    \end{align}
    """

    def __init__(self, d_model: int, token_dim: (int,int), local_window_size: (int,int), bias: bool = True):
        """
        * `d_model` is the number of features in the `query`, `key` and `value` vectors.
        * `token_dim` is image size like $(32,64)$
        * `local_window_size` is the local window size $s$
        * `bias` is whether to have a bias parameter for transformations for $Q$, $K$ and $V$.
        # input is a (B,head,W,H,embedding) tensor
        # in each head dimension
        #    do 
        """

        super().__init__()
        # Local window size $s$
        self.local_window_size = local_window_size
        # These transform the `query`, `key` and `value` vectors.
        self.query      = nn.Linear(d_model, d_model, bias=bias)
        self.key        = nn.Linear(d_model, d_model, bias=bias)
        self.value      = nn.Linear(d_model, d_model, bias=bias)
        self.pos_bias   = nn.Parameter(torch.zeros(seq_len, seq_len), requires_grad=True)
        self.local_mask = nn.Parameter(self.create_local_mask(token_dim, local_window_size), requires_grad=False)
        self.activation = nn.Sigmoid()
        self.output     = nn.Linear(d_model, d_model)

    @staticmethod
    def create_local_mask(token_dim, local_window_size):
        """
        if the token_dim is seq_len then produce (seq_len,seq_len)
        #### Create local mask
        This creates a mask for
        \begin{align}
        m_{t,t'} =
        \begin{cases}
        1,  & {\text{for } \lvert t-t' \rvert \lt s} \\
        0, & \text{otherwise}
        \end{cases}
        \end{align}
        """

        # Initialize to ones
        local_mask = torch.ones(seq_len, seq_len, dtype=torch.bool)
        # Make $t' - t \ge s$ zero
        local_mask = torch.tril(local_mask, local_window_size - 1)
        # Make $t - t' \ge s$ zero
        local_mask = torch.triu(local_mask, -(local_window_size - 1))

        #
        return local_mask

    def forward(self, *,
                query: torch.Tensor,
                key: torch.Tensor,
                value: torch.Tensor,
                mask: Optional[torch.Tensor] = None):
        """
        `query`, `key` and `value` are the tensors that store
        collection of token embeddings for  *query*, *key* and *value*.
        They have shape `[seq_len, batch_size, d_model]`.
        `mask` has shape `[seq_len, seq_len, batch_size]` and
        `mask[i, j, b]` indicates whether for batch `b`,
        query at position `i` has access to key-value at position `j`.
        """

        # `query`, `key` and `value`  have shape `[B,head,W,H,P]`
        B,head,W,H,P = query.shape

        if mask is not None:
            # `mask` has shape `[seq_len_q, seq_len_k, batch_size]`,
            # where first dimension is the query dimension.
            # If the query dimension is equal to $1$ it will be broadcasted.
            assert mask.shape[0] == 1 or mask.shape[0] == query.shape[0]
            assert mask.shape[1] == key.shape[0]
            assert mask.shape[2] == 1 or mask.shape[2] == query.shape[1]

        # Transform query, key and value embeddings
        query   = self.query(query) # (B,head,W,H,P)
        key     = self.key(key)     # (B,head,W,H,P)
        value   = self.value(value) # (B,head,W,H,P)
        max_key = key.max(dim=0, keepdims=True)[0]
        key     = torch.exp(key - max_key)
        
        kv  = key * value # (B,head,P,W,H)
        
        convkvw = torch.nn.functional.conv2d(rearrange(kv," B D P W H -> (B D) P W H "),# (B*head,P,W,H)
                                             self.)
        num =  + torch.einsum('bdwhp,bdwhp->bdp', exp_key, value)
        # The denominator part $\sum_{t'=1}^T \exp(w_{t,t'}) \odot \exp(K_{t'})$
        den = torch.einsum('ijb,jbd->ibd', exp_pos_bias, exp_key)

        # Output $$Y_t = \sigma(Q_t) \odot
        #         \frac{\sum_{t'=1}^T \exp(w_{t,t'}) \odot \exp(K_{t'}) \odot V_{t'}}
        #         {\sum_{t'=1}^T \exp(w_{t,t'}) \odot \exp(K_{t'})}$$
        y = self.activation(query) * num / den

        # Output layer
        return self.output(y)


def _test_local_mask():
    """
    Test local mask
    """
    from labml.logger import inspect
    inspect(AFTLocal.create_local_mask(10, 4))


#
if __name__ == '__main__':
    _test_local_mask()

In [2]:
from model.custom_transformer import *

In [6]:
from model.FEDformer import *

In [14]:
kargs={'img_size': (7, 7), 'patch_size': 1, 'patch_range': 7, 'in_chans': 73, 'out_chans': 70, 'fno_blocks': 4, 'embed_dim': 512, 'depth': 2, 'debug_mode': 0, 'double_skip': False, 'fno_bias': False, 'fno_softshrink': 0.0, 'history_length': 16, 'reduce_Field_coef': False, 'modes': (17, 33, 6), 'mode_select': 'normal', 'physics_num': 4, 'pred_len': 16, 'n_heads': 8, 'label_len': 8, 'canonical_fft': 1, 'unique_up_sample_channel': 70, 'share_memory': 0, 'dropout_rate': 0}

In [15]:
model = FEDformer(**kargs)

fourier enhanced block used!
for shape:[7 7] and pick modes:(17, 33)>=25.0, we pick 25.0 modes, the baseline modes is 25.0
create a mode filter shape=torch.Size([7, 7, 9]) with 150 mode activate
fourier enhanced block used!
for shape:[7 7] and pick modes:(17, 33)>=25.0, we pick 25.0 modes, the baseline modes is 25.0
create a mode filter shape=torch.Size([7, 7, 13]) with 150 mode activate
 fourier enhanced cross attention used!
for shape:[7 7] and pick modes:(17, 33)>=25.0, we pick 25.0 modes, the baseline modes is 25.0
for shape:[7 7] and pick modes:(17, 33)>=25.0, we pick 25.0 modes, the baseline modes is 25.0
 modes_q=150,  shape_q=torch.Size([7, 7, 13])
modes_kv=150, shape_kv=torch.Size([7, 7, 9])


In [16]:
from mltool.universal_model_util import get_model_para_detail

In [17]:
i=0
j=0
data = []
for name,p in model.named_parameters():
    data.append([name,np.prod(p.shape),p.detach().numpy().shape])
    #print("{:40} {:5} {}".format(name,np.prod(p.shape),p.shape))
    i+=np.prod(p.shape)
    j+=1
print("Totally {} parameters, totally {} numbers".format(j,i))

Totally 74 parameters, totally 17801286 numbers


In [18]:
import pandas as pd

In [19]:
pdata = pd.DataFrame(data,columns=['weight','size','shape'])

In [20]:
pdata.sort_values('size')

,weight,size,shape
73,decoder.projection.bias,70,"(70,)"
27,encoder.norm.batchnorm.bias,512,"(512,)"
30,decoder.layers.0.self_attention.query_projecti...,512,"(512,)"
32,decoder.layers.0.self_attention.key_projection...,512,"(512,)"
34,decoder.layers.0.self_attention.value_projecti...,512,"(512,)"
...,...,...,...
40,decoder.layers.0.cross_attention.key_projectio...,262144,"(512, 512)"
11,encoder.attn_layers.0.attention.out_projection...,262144,"(512, 512)"
59,decoder.layers.1.cross_attention.query_project...,262144,"(512, 512)"
37,decoder.layers.0.cross_attention.inner_correla...,4915200,"(8, 64, 64, 150)"


In [6]:
from utils.tools import getModelSize

In [7]:
getModelSize(model)

(51232766, 4098, 195.45313262939453)

In [20]:
x_enc     = torch.randn(1,32,64,6,70).cuda()
x_mark_enc= torch.randn(1,6,4).cuda()
x_mark_dec= torch.randn(1,1,4).cuda()
model(x_enc,x_mark_enc,x_mark_dec).shape

torch.Size([1, 32, 64, 6, 70])

In [21]:
%%timeit
with torch.no_grad():
    with torch.cuda.amp.autocast(enabled=False):
        model(x_enc,x_mark_enc,x_mark_dec).shape

811 ms ± 20.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
with torch.no_grad():
    with torch.cuda.amp.autocast(enabled=True):
        model(x_enc,x_mark_enc,x_mark_dec).shape

696 ms ± 8.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
q = torch.randn(1,2,3,4,5)
k = torch.randn(1,2,3,4,5)
v = torch.randn(1,2,3,4,5)

In [29]:
layer(q,k,v,None)[0].shape

torch.Size([1, 2, 3, 4, 5])

In [3]:
model = POverLapTimePosVallinaViT((14,32,64),(3,5,5),in_chans=5)

for 2D patch model, the img_size will be force set (14, 32, 64)


In [4]:
from train.pretrain import *

In [5]:
args = get_args(config_path="checkpoints/WeathBench7066PatchDataset/ViT_in_bulk-POverLapTimePosVallinaViT_Patch_(3, 5, 5)/time_step_2_pretrain-3D70N_every_1_step_random_dataset/12_07_22_50_36-seed_2018/config.json")

In [6]:
time_step = args.time_step if "fourcast" in args.mode else 3*24//6 + args.time_step
dataset_kargs = copy.deepcopy(args.dataset_kargs)
dataset_kargs['time_step'] = time_step
if dataset_kargs['time_reverse_flag'] in ['only_forward','random_forward_backward']:
    dataset_kargs['time_reverse_flag'] = 'only_forward'
dataset_type = eval(args.dataset_type) if isinstance(args.dataset_type,str) else args.dataset_type
#print(dataset_kargs)
split = args.split if hasattr(args,'split') and args.split else "test"
test_dataset = dataset_type(split=split, with_idx=True,dataset_tensor=None,record_load_tensor=None,**dataset_kargs)

use offline data mode <1>: only train use offline data
use dataset in datasets/weatherbench_6hour
load data from datasets/weatherbench_6hour/test.npy
notice we will use around_index(12, 28, 64, 3, 3, 5, 5) to patch data


In [18]:
test_dataset.cross_sample = False
idx_a,batch = test_dataset[0]
tensor_a, time_stamp_a, pos_a = batch[0]
input_a = model.image_to_patches(tensor_a[None],pos_a)

In [29]:
x = tensor_a[None]
good_input_shape = model.patch_range
now_input_shape  = tuple(x.shape[-3:])

around_index = model.around_index_depend_input(now_input_shape)
if model.global_position_feature is None:
    grid = torch.Tensor(np.stack(np.meshgrid(
            np.arange(model.img_size[0]),
            np.arange(model.img_size[1]),
            np.arange(model.img_size[2]))).transpose(0,2,1,3))
    model.global_position_feature = model.get_direction_from_stamp(grid[None]) #(1, 3, Z,W, H)
pos_feature  = model.global_position_feature.repeat(x.size(0),1,1,1,1).to(x.device) #(B, 3, Z,W, H)
B,P,_,_,_ = x.shape
x = torch.cat([x,pos_feature],1) #( B, 77, W, H)
# x = x[...,around_index[:,:,:,0],around_index[:,:,:,1],around_index[:,:,:,2]] # (B,P,Z,W-4,H,Patch,Patch)
# B,_,Z,W,H,_,_,_ = x.shape
# model.input_shape_tmp=(B,Z,W,H,P)

In [32]:
pos_b.shape

(3, 3, 5, 5)

In [41]:
test_dataset.cross_sample = True
idx_b,batch = test_dataset[0]
(tensor_b, time_stamp_b, pos_b) = batch[0]
input_b = model.image_to_patches(tensor_b[None],torch.Tensor(pos_b[None]).to(tensor_b.device))
print(pos_b)

[[[[10 10 10 10 10]
   [10 10 10 10 10]
   [10 10 10 10 10]
   [10 10 10 10 10]
   [10 10 10 10 10]]

  [[11 11 11 11 11]
   [11 11 11 11 11]
   [11 11 11 11 11]
   [11 11 11 11 11]
   [11 11 11 11 11]]

  [[12 12 12 12 12]
   [12 12 12 12 12]
   [12 12 12 12 12]
   [12 12 12 12 12]
   [12 12 12 12 12]]]


 [[[19 19 19 19 19]
   [20 20 20 20 20]
   [21 21 21 21 21]
   [22 22 22 22 22]
   [23 23 23 23 23]]

  [[19 19 19 19 19]
   [20 20 20 20 20]
   [21 21 21 21 21]
   [22 22 22 22 22]
   [23 23 23 23 23]]

  [[19 19 19 19 19]
   [20 20 20 20 20]
   [21 21 21 21 21]
   [22 22 22 22 22]
   [23 23 23 23 23]]]


 [[[18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]]

  [[18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]]

  [[18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]
   [18 19 20 21 22]]]]


In [42]:
selected_tensor= x[...,pos_b[0],pos_b[1],pos_b[2]]
torch.dist(selected_tensor,input_b)

tensor(0., dtype=torch.float64)

In [19]:
grid = torch.Tensor(np.stack(np.meshgrid(
                np.arange(model.img_size[0]),
                np.arange(model.img_size[1]),
                np.arange(model.img_size[2]))).transpose(0,2,1,3))

In [23]:
a = torch.randn(1,5,14,32,64)
y = model.image_to_patches(a)
c = model.patches_to_image(y)

TypeError: image_to_patches() missing 1 required positional argument: 'pos_stamp'

In [20]:
grid.shape

torch.Size([3, 14, 32, 64])

In [8]:
a = torch.randn(1,70,3,5,5).cuda()
p = torch.randn(1, 3,3,5,5).cuda()
t = torch.randn(1,4).cuda()
model = model.cuda()

In [9]:
model(a,t,p).shape

torch.Size([1, 70, 3, 5, 5])

In [7]:
import numpy as np

In [18]:
torch.Tensor(np.stack(np.meshgrid(
            np.arange(img_size[0]),
            np.arange(img_size[1]),
        )).transpose(0,2,1)).shape

torch.Size([2, 28, 32])

In [19]:
torch.Tensor(np.stack(np.meshgrid(
    np.arange(self.img_size[0]),
    np.arange(self.img_size[1]),
    np.arange(self.img_size[2]))).transpose(0,2,1,3)).shape


torch.Size([3, 28, 32, 36])

In [4]:
model(a).shape

torch.Size([1, 70, 14, 32, 64])

In [6]:
32*16*7

3584

In [101]:
a = torach.randn(1,2,4,4)
m = torch.randint(2,(4,4))

In [102]:
a.masked_fill(m,-1)

tensor([[[[-1.0000, -1.0000,  1.2643,  0.2839],
          [-1.0000,  0.0204, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -0.7041],
          [ 0.1266, -1.0000, -1.0000, -1.0000]],

         [[-1.0000, -1.0000, -2.2420, -0.1599],
          [-1.0000,  1.4979, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -0.3242],
          [ 0.3754, -1.0000, -1.0000, -1.0000]]]])

In [2]:
import numpy as np

In [ ]:

#x = x.reshape(B,W,H,P,patch_range,patch_range)
x = torch.randn(B,W,H,P,patch_range,patch_range)
def get_x1(x,counting_matrix):
    w_idx   = np.arange(0,L)
    wes  = np.stack([w_idx, w_idx+1,w_idx+2, w_idx-1, w_idx-2],1)%L
    x_idx   = np.arange(H)
    xes = np.stack([x_idx, x_idx+1,x_idx+2, x_idx-1, x_idx-2],1)%H
    yes = np.array([[2,  1,  0,  3,  4]])
    x = torch.nn.functional.pad(x,(0,0, 0,0, 0,0, 0,0, 2 , 2 )) # only extend W
    x     = x[:, wes, :,:,yes,:].sum(1) #(B, W, H, P, PS,PS) --> (W, B, H, P, PS)
    x     = x[:, :, xes,:,yes].sum(1)  #(W, B, H, P, PS) --> (H, W, B, P)   
    x     = x.permute(2,3,1,0)#(B,P, W,H)   
    counting_matrix =counting_matrix.to(x.device)
    x     = x/counting_matrix #(B,P, W,H)  / (1,1 , W,H)
    return x
def get_x2(x,counting_matrix):
    w_idx   = np.arange(0,L)
    wes  = np.stack([w_idx+2, w_idx+1, w_idx, w_idx-1, w_idx-2],1)%L
    x_idx   = np.arange(H)
    xes = np.stack([x_idx+2, x_idx+1, x_idx, x_idx-1, x_idx-2],1)%H
    yes = np.array([[0 ,1, 2, 3, 4]])
    x = torch.nn.functional.pad(x,(0,0, 0,0, 0,0, 0,0, 2 , 2 )) # only extend W
    x     = x[:, wes, :,:,yes,:].sum(1) #(B, W, H, P, PS,PS) --> (W, B, H, P, PS)
    x     = x[:, :, xes,:,yes].sum(1)  #(W, B, H, P, PS) --> (H, W, B, P)   
    x     = x.permute(2,3,1,0)#(B,P, W,H)   
    counting_matrix =counting_matrix.to(x.device)
    x     = x/counting_matrix #(B,P, W,H)  / (1,1 , W,H)
    return x
print(torch.dist(get_x1(x,counting_matrix),get_x2(x,counting_matrix)))

In [ ]:
from vit_pytorch.vit import repeat

In [ ]:
from mltool.universal_model_util import get_model_para_num
from utils.tools import getModelSize

In [ ]:
from model.othermodels import *

In [ ]:
class CK_SWDformer_0505(BaseModel):
        def __init__(self,*args,**kargs):
            super().__init__()
            print("this is pre-set model, we disable all config")
            self.backbone = SWD_former(patch_size= [1, 1],
                                in_chans= 70,
                                out_chans= 70,
                                embed_dim= 768,
                                window_size= (5,5),
                                depths= [4, 4, 4],
                                num_heads= [6, 6, 6],
                                Weather_T=1,only_swin=True)
        def forward(self,x):
            return self.backbone(x)

In [ ]:
model = CK_SWDformer_0505()

In [ ]:
a=torch.randn(1,70,5,5)
model(a).shape

In [ ]:
W=32
H=64
hd=16
D=768
B=3

In [ ]:
from networks.SWD_former import SWD_former,SWDfromer

In [ ]:
model = SWD_former(patch_size= [1, 1],
        in_chans= 70,
        out_chans= 70,
        embed_dim= 1128,
        window_size= (32,64),
        depths= [4, 4, 4],
        num_heads= [6, 6, 6],
        Weather_T=1,only_swin=True)

In [ ]:
from mltool.universal_model_util import get_model_para_num

In [ ]:
from model.afnonet import AFNONet

In [ ]:
get_model_para_num(model)[1]/1024/1024

In [ ]:
import torch

In [ ]:
import numpy as np

In [ ]:
from cephdataset import WeathBench7066PatchDataset

In [ ]:
img = torch.randn(1,70,32,64)
x = model.backbone.to_patch_embedding(img);print(x.shape)
b, n, _ = x.shape

cls_tokens = repeat(model.backbone.cls_token, '1 1 d -> b 1 d', b = b)
x = torch.cat((cls_tokens, x), dim=1);print(x.shape)
x += model.backbone.pos_embedding[:, :(n + 1)];print(x.shape)
x = model.backbone.dropout(x);print(x.shape)

x = model.backbone.transformer(x);print(x.shape)

x = x.mean(dim = 1) if model.backbone.pool == 'mean' else x[:, 0];print(x.shape)

x = model.backbone.to_latent(x);print(x.shape)

In [ ]:
property_names = test_dataset.vnames
accu_list = torch.stack([p['accu'] for p in fourcastresult.values() if 'accu' in p]).numpy()
total_num = len(accu_list)
accu_list = accu_list.mean(0)# (fourcast_num,property_num)
real_times = [(predict_time+1)*test_dataset.time_intervel*test_dataset.time_unit for predict_time in range(len(accu_list))]

In [ ]:
snap_tables=[]
snap_index = fourcastresult['snap_index']
select_snap_start_time_point = snap_index[0]
select_snap_show_property_id = snap_index[1]
select_snap_property_name    = [property_names[iidd] for iidd in select_snap_show_property_id]
for select_time_point in select_snap_start_time_point:
    timestamp = test_dataset.datatimelist_pool['test'][select_time_point]
    if select_time_point in fourcastresult: # in case do not record
        linedata = fourcastresult[select_time_point]['snap_line']
        for predict_time_point, tensor, label in linedata:
            for name, value in zip(select_snap_property_name,tensor):
                predict_timestamp = 0 if predict_time_point==0 else real_times[predict_time_point-1]
                snap_tables.append([timestamp, name, predict_timestamp, value.item(), label])

In [ ]:
global_rmse_map = fourcastresult['global_rmse_map']
mean_global_rmse_map = [t/total_num for t in global_rmse_map]

In [ ]:
import numpy 

In [ ]:
import pandas as pd

# Old

In [2]:
import torch
 

In [3]:
model  = torch.nn.Linear(32,64)

In [5]:
optimizer       = torch.optim.AdamW(model.parameters())

In [7]:
optimizer.param_groups[0]['foreach']

In [ ]:
class LargeMLP_3D(nn.Module):
    '''
    input is (B, P, patch_range_1,patch_range_2,patch_range_3)
    output is (B,P)
    ''' 
    def __init__(self,img_size=None,patch_range=5,in_chans=20, out_chans=20,p=0.1,**kargs):
        super().__init__()
        self.img_size = img_size
        self.patch_range = 5
        if self.patch_range == 5:
            cl = [5*5*5*in_chans,5*5*5*10,5*5*5*20,
                  5*5*5*30,5*5*5*30,5*5*5*20,
                  5*5*5*10,5*5*5*1,out_chans]
            nnlist = []
            for i in range(len(cl)-2):
                nnlist+=[nn.Linear(cl[i],cl[i+1]),nn.Dropout(p=p),nn.Tanh()]
            nnlist+=[nn.Linear(cl[-2],cl[-1])]
            self.backbone = nn.Sequential(*nnlist)
        else:
            raise NotImplementedError
        self.center_index,self.around_index=get_center_around_indexes_3D(self.patch_range,self.img_size)

    def forward(self, x):
        '''
        The input either (B,P,patch_range,patch_range) or (B,P,w,h)
        The output then is  (B,P) or (B,P,w-patch_range//2,h-patch_range//2)
        ''' 
        assert len(x.shape) == 5 #(B,P,Z,W,H)
        input_is_full_image = False
        if x.shape[-3:] == self.img_size:
            input_is_full_image = True
            x = x[..., self.around_index[:, :, : , 0],self.around_index[:, :, : , 1],self.around_index[:, :, : , 2]] 
            # (B,P,Z-2,W-2,H,Patch,Patch,Patch)
            x = x.permute(0, 2, 3, 4, 1, 5, 6, 7)
            B, Z, W, H, P, _, _, _ = x.shape
            x = x.flatten(0, 3)  # (B* Z-2 * W-2 * H, Property, Patch,Patch,Patch)
        assert tuple(x.shape[-3:]) == (self.patch_range,self.patch_range,self.patch_range)
        x = self.backbone(x.flatten(-4,-1)) # (B* W-4 * H,P)
        if input_is_full_image:
            x = x.reshape(B, Z, W, H, P).permute(0, 4, 1, 2,3) #(B, Z-2,W-2,H,P)  -> (B,P, Z-2,W-2,H)
        return x


In [ ]:
model = LargeMLP_3D((14,32,64))

In [ ]:
from utils.tools import getModelSize
param_sum, buffer_sum, all_size = getModelSize(model)
print(f" Number of Parameters: {param_sum}, Number of Buffers: {buffer_sum}, Size of Model: {all_size:.4f} MB\n")

In [ ]:
from model.patch_model import NaiveConvModel2D

In [ ]:
import torch

In [ ]:
model = NaiveConvModel2D((32,64))

In [ ]:
a = torch.randn(1,20,32,64)

In [ ]:
from utils.tools import get_center_around_indexes

In [ ]:
img_shape=(32,64)
patch_range=5
center_index,around_index=get_center_around_indexes(patch_range,img_shape)

In [ ]:
around_index.shape

In [ ]:
patch_range=5
delta = [list(range(-(patch_range//2),patch_range//2+1))]*2
delta = np.meshgrid(*delta)

In [ ]:
delta[0].shape

In [ ]:
delta = [list(range(-(patch_range//2),patch_range//2+1))]*len(center)
delta = np.meshgrid(*delta)
pos  = [c+dc for c,dc in zip(center,delta)]
pos[-1]= pos[-1]%64

px = x + dx
py = y + dy
np.stack([px.flatten(),py.flatten()])

In [ ]:
import torch

In [ ]:
coordinate = np.arange(36).reshape(1,1,6,6)
print(coordinate)

In [ ]:
coordinate[:,:,pos].shape

In [ ]:
print(coordinate[:,:,x,y][0,0])
print(coordinate[:,:,pos[0],pos[1]][0,0])

In [ ]:
from model.FEDformer import *

In [ ]:
nn.BatchNorm3d()

In [ ]:
class moving_avg_spacetime(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super().__init__()
        assert len(kernel_size)==3
        self.kernel_size = np.array(kernel_size)
        self.avg         = nn.AvgPool3d(kernel_size=kernel_size, stride=stride, padding=0)
        self.pad_front   = self.kernel_size - 1-np.floor((self.kernel_size - 1) // 2)
        self.pad_end     = np.floor((self.kernel_size - 1) // 2)
        self.pad         = np.stack([self.pad_front,self.pad_end],1)[::-1].flatten().astype('int').tolist()
        print(self.pad)
    def forward(self, x):
        # padding on the both ends of time series
        # the input must be (B,*Space,T,C)， the -1 dim is embed channel, the -2 dim is time channel
        shape = x.shape
        BSpace_shape = shape[:-2]
        C = shape[-1]
        permute_order = [0,-1] + list(range(1,len(shape)-1))
        x = x.permute(*permute_order)#-->(B, *Space,T, C)-->(B, C, *Space,T)
        x = self.avg(F.pad(x,self.pad, mode='replicate'))
        permute_order = [0] + list(range(2,len(shape))) + [1]
        x = x.permute(*permute_order)
        return x

In [ ]:
layer = moving_avg_spacetime((5,3,2),1)

In [ ]:
a=torch.randn(1,32,64,6,7)
layer(a).shape

In [ ]:
from model.physics_model import *

In [ ]:
from model.afnonet import *

In [ ]:
from utils.params import *

In [ ]:
args=get_args(args=[])

In [ ]:
backbone = AFNONet((32,64),2,265,20,depth=1)

In [ ]:
layer = DirectSpace_Feature_Model(args,backbone)

In [ ]:
a= torch.randn(1,4,5,32,64)

In [ ]:
torch.std_mean(layer(a))

In [ ]:
layer = Second_Derivative_Layer()

a=torch.randn(1,1,32,64)

layer(a).shape

In [ ]:
B=2
P=4
a=torch.randn(B,P,3,32,64).cuda()
layer=  First_Derivative_Layer(dim=3).cuda()
runtime_weight=layer.runtime_weight

In [ ]:
%%timeit
x = torch.conv3d(a.flatten(0,1).unsqueeze(1),runtime_weight).reshape(*a.shape[:-1],-1)

In [ ]:
%%timeit
x2 = torch.conv1d(a.flatten(0,-2).unsqueeze(1),runtime_weight[0,0]).reshape(*a.shape[:-1],-1)

In [ ]:
torch.dist(x,x2)

In [ ]:
%matplotlib inline
plt.scatter(x,y)
#plt.scatter([10],[10],'r')

In [ ]:
def get_fourier_drive(x, position,pad):

    if len(position)==1:position = position[0]
    base_len          = np.array(x.shape[position])
    interpolate_shape = base_len + (base_len-1)*pad
    interpolate_shape = tuple(interpolate_shape)
    if isinstance(position,int) or len(position) == 1
        x = x.transpose(-1,position)
        out_shape = x.shape[:-1]
        x = x.reshape(x.size(0),-1,x.size(-1))
        b = torch.nn.functional.interpolate(x,interpolate_shape,mode='linear',align_corners=True)
        bf = torch.fft.rfft(b,dim=-1)
        kbf= bf*torch.fft.rfftfreq(b.size(-1)).reshape(1,1,-1)
        kb = torch.fft.irfft(kbf,dim=-1)
        index = torch.arange(0,b.size(-1)+1,base_len-1)
        kb = kb[...,index]

In [ ]:
#from tkinter.messagebox import NO
import numpy as np
import torch,os,io,socket
from torchvision import datasets, transforms
hostname = socket.gethostname()
from functools import lru_cache
import traceback
from tqdm import tqdm
import pandas as pd
from utils.timefeatures import time_features
import os
import h5py


In [ ]:
datatimelist  = np.arange(np.datetime64("1979-01-01"), np.datetime64("2016-01-01"), np.timedelta64(6, "h"))
timestamp = time_features(pd.to_datetime(datatimelist)).transpose(1, 0)


In [ ]:
pd.to_datetime(np.datetime64("1980-12-31")).dayofyear

In [ ]:
pd.to_datetime(datatimelist)[0].dayofyear

In [ ]:
timestamp[:10,[0,-1]]

In [ ]:
from JCmodels.fourcastnet import *

In [ ]:
model = AFNONet((32,64))

In [ ]:
a = torch.randn(1,4,5,32,64)

In [ ]:
model(a).shape

#### complex version AdaptiveFourierNeuralOperator

##### validate

In [ ]:
def multiply(input, weights):
    return torch.einsum('...bd,bdk->...bk', input, weights)

In [ ]:
B=1
h=32;w=64;C=2
num_blocks=1
block_size=2

w1 = torch.randn(num_blocks, block_size, block_size, dtype=torch.cfloat)
b1 = torch.randn(num_blocks, block_size, dtype=torch.cfloat)
w2 = torch.randn(num_blocks, block_size, block_size, dtype=torch.cfloat)
b2 = torch.randn(num_blocks, block_size, dtype=torch.cfloat)

In [ ]:
def original_realize(x,w1,b1,w2,b2):
    x = torch.fft.rfft2(x, dim=(1, 2), norm='ortho');
    x = x.reshape(B, x.shape[1], x.shape[2], num_blocks, block_size)
    x_real = F.relu(multiply(x.real, w1.real) - multiply(x.imag, w1.imag) + b1.real, inplace=True)
    x_imag = F.relu(multiply(x.real, w1.imag) + multiply(x.imag, w1.real) + b1.imag, inplace=True)
    x = torch.stack([x_real, x_imag], dim=-1)
    x = torch.view_as_complex(x)
    return x

def complex_version_realize(x,w1,b1,w2,b2):
    x = torch.fft.rfft2(x, dim=(1, 2), norm='ortho');
    x = x.reshape(B, x.shape[1], x.shape[2], num_blocks, block_size)
    x = multiply(x,w1)+b1
    x = nonlinear_activate(x)
    return x
x  = torch.randn(B, h, w, C)
y1 = original_realize(x,w1,b1,w2,b2)
y2 = complex_version_realize(x,w1,b1,w2,b2)
torch.dist(y1,y2)

In [ ]:
from model.afnonet import *

##### AFNONet3D

In [ ]:
class PartReLU_Complex(nn.Module):
    def forward(self,x):
        F.relu(x.real, inplace=True)
        F.relu(x.imag, inplace=True)
        return x

class AdaptiveFourierNeuralOperator(nn.Module):
    def __init__(self, dim, img_size, fno_blocks=4,fno_bias=True, fno_softshrink=False,nonlinear_activate=PartReLU_Complex()):
        super().__init__()

        self.hidden_size = dim
        self.img_size   = img_size
        self.num_blocks = fno_blocks
        self.block_size = self.hidden_size // self.num_blocks
        assert self.hidden_size % self.num_blocks == 0

        self.scale = 0.02
        self.w1 = torch.nn.Parameter(self.scale * torch.randn(self.num_blocks, self.block_size, 
                                                              self.block_size, dtype=torch.cfloat))
        self.b1 = torch.nn.Parameter(self.scale * torch.randn(self.num_blocks, self.block_size, 
                                                              dtype=torch.cfloat))
        self.w2 = torch.nn.Parameter(self.scale * torch.randn(self.num_blocks, self.block_size, 
                                                              self.block_size, dtype=torch.cfloat))
        self.b2 = torch.nn.Parameter(self.scale * torch.randn(self.num_blocks, self.block_size, 
                                                              dtype=torch.cfloat))
        self.relu = nonlinear_activate

        if fno_bias:
            self.bias = nn.Conv1d(self.hidden_size, self.hidden_size, 1)
        else:
            self.bias = None

        self.softshrink = fno_softshrink

    def multiply(self, input, weights):
        return torch.einsum('...bd,bdk->...bk', input, weights)

    def forward(self, x):
        B, N, C = x.shape    
        bias = self.bias(x.permute(0, 2, 1)).permute(0, 2, 1) if self.bias else 0
        #timer.restart(2)
        x = x.reshape(B, *self.img_size, C)
        fft_dim = tuple(range(1,len(self.img_size)+1))
        print(fft_dim)
        #timer.record('reshape1','filter',2)
        print(x.shape)
        x = torch.fft.rfftn(x, dim=fft_dim, norm='ortho');
        #timer.record('rfft2','filter',2)
        print(x.shape)
        x = x.reshape(*x.shape[:-1], self.num_blocks, self.block_size)
        #timer.record('reshape2','filter',2)
        x = self.multiply(x,self.w1)+self.b1
        x = self.relu(x)
        x = self.multiply(x,self.w2)+self.b2
        x = F.softshrink(x, lambd=self.softshrink) if self.softshrink else x
        #with torch.cuda.amp.autocast(enabled=False):
        #x = x.float()   
        #x = torch.view_as_complex(x)
        #timer.record('reset','filter',2)
        x = x.flatten(-2,-1)
        #timer.record('reshape3','filter',2)
        print(x.shape)
        x = torch.fft.irfftn(x, s=self.img_size,dim=fft_dim, norm='ortho')
        print(x.shape)
        #x = x.half()
        #timer.record('irfft2','filter',2)
        x = x.reshape(B, N, C)
        #timer.record('reshape4','filter',2)
        return x + bias

class Block(nn.Module):
    def __init__(self, dim, mlp_ratio=4., drop=0., drop_path=0., act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm, region_shape=(14,8), fno_blocks=3,double_skip=False, fno_bias=False, fno_softshrink=False):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.filter = AdaptiveFourierNeuralOperator(dim, region_shape,fno_blocks=fno_blocks,fno_bias=fno_bias,fno_softshrink=fno_softshrink)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        self.double_skip = double_skip

    def forward(self, x):
        residual = x
        #timer.restart(1)
        x = self.norm1(x)
        #timer.record('norm1','forward_features',1)
        x = self.filter(x)
        #timer.record('filter','forward_features',1)
        if self.double_skip:
            x += residual
            residual = x;
        #timer.record('residual','forward_features',1)
        x = self.norm2(x)
        #timer.record('norm2','forward_features',1)
        x = self.mlp(x)
        #timer.record('mlp','forward_features',1)
        x = self.drop_path(x)
        #timer.record('drop_path','forward_features',1)
        x += residual
        #timer.record('residual','forward_features',1)
        return x

class PatchEmbed(nn.Module):
    def __init__(self, img_size=None, patch_size=8, in_chans=13, embed_dim=768):
        super().__init__()

        if img_size is None:raise KeyError('img is None')
        patch_size   = [patch_size]*len(img_size) if isinstance(patch_size,int) else patch_size
        
        num_patches=1
        out_size=[]
        for i_size,p_size in zip(img_size,patch_size):
            if p_size%i_size:
                num_patches*=i_size// p_size
                out_size.append(i_size// p_size)
            else:
                raise NotImplementedError(f"the patch size ({patch_size}) cannot divide the img size {img_size}")
        self.img_size    = tuple(img_size)
        self.patch_size  = tuple(patch_size)
        self.num_patches = num_patches
        self.out_size    = tuple(out_size)
        conv_engine = [nn.Conv1d,nn.Conv2d,nn.Conv3d]
        self.proj   = conv_engine[len(img_size)-1](in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, = x.shape[:2]
        inp_size = x.shape[2:]
        assert tuple(inp_size) == self.img_size, f"Input image size ({inp_size}) doesn't match model set size ({self.img_size})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

class AFNONet(nn.Module):
    """
    
    """
    def __init__(self, img_size, patch_size=8, in_chans=20, out_chans=20, embed_dim=768, depth=12, mlp_ratio=4.,
                 uniform_drop=False, drop_rate=0., drop_path_rate=0., norm_layer=None,
                 dropcls=0, checkpoint_activations=False, fno_blocks=3,double_skip=False,
                 fno_bias=False, fno_softshrink=False,debug_mode=False):
        super().__init__()

        assert img_size is not None
        self.checkpoint_activations=checkpoint_activations
        self.embed_dim = embed_dim
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        self.img_size = img_size
        self.patch_embed = PatchEmbed(img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches      = self.patch_embed.num_patches
        patch_size       = self.patch_embed.patch_size
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        self.final_shape = self.patch_embed.out_size

        if uniform_drop:
            dpr = [drop_path_rate for _ in range(depth)]  # stochastic depth decay rule
        else:
            dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule

        self.blocks = nn.ModuleList([Block(dim=embed_dim, mlp_ratio=mlp_ratio, drop=drop_rate,
                                           drop_path=dpr[i], 
                                           norm_layer=norm_layer,
                                           region_shape=self.final_shape,
                                           double_skip=double_skip,
                                           fno_blocks=fno_blocks,
                                           fno_bias=fno_bias,
                                           fno_softshrink=fno_softshrink) for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        # Representation layer
        # self.num_features = out_chans * img_size[0] * img_size[1]
        # self.representation_size = self.num_features * 8
        # self.pre_logits = nn.Sequential(OrderedDict([
        #     ('fc', nn.Linear(embed_dim, self.representation_size)),
        #     ('act', nn.Tanh())
        # ]))
        conf_list = [{'kernel_size':[],'stride':[],'padding':[]},
                     {'kernel_size':[],'stride':[],'padding':[]},
                     {'kernel_size':[],'stride':[],'padding':[]}]
        conv_set = {8:[[2,2,0],[2,2,0],[2,2,0]],
                    4:[[2,2,0],[3,1,1],[2,2,0]],
                    2:[[3,1,1],[3,1,1],[2,2,0]],
                    1:[[3,1,1],[3,1,1],[3,1,1]],
                   }
        for patch in patch_size:
            for slot in range(len(conf_list)):
                conf_list[slot]['kernel_size'].append(conv_set[patch][slot][0])
                conf_list[slot]['stride'].append(conv_set[patch][slot][1])
                conf_list[slot]['padding'].append(conv_set[patch][slot][2])

        transposeconv_engine = [nn.ConvTranspose1d,nn.ConvTranspose2d,nn.ConvTranspose3d][len(img_size)-1]
        self.pre_logits = nn.Sequential(OrderedDict([
            ('conv1', transposeconv_engine(embed_dim, out_chans*16, **conf_list[0])),
            ('act1', nn.Tanh()),
            ('conv2', transposeconv_engine(out_chans*16, out_chans*4, **conf_list[1])),
            ('act2', nn.Tanh())
        ]))

        # Generator head
        # self.head = nn.Linear(self.representation_size, self.num_features)
        self.head = transposeconv_engine(out_chans*4, out_chans, **conf_list[2])

        if dropcls > 0:
            print('dropout %.2f before classifier' % dropcls)
            self.final_dropout = nn.Dropout(p=dropcls)
        else:
            self.final_dropout = nn.Identity()

        trunc_normal_(self.pos_embed, std=.02)
        self.apply(self._init_weights)
        self.debug_mode=debug_mode
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def forward_features(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        x += self.pos_embed
        x = self.pos_drop(x)

        if not self.checkpoint_activations:
            for blk in self.blocks:
                x = blk(x)
        else:
            x = checkpoint_sequential(self.blocks, 4, x)

        x = self.norm(x).transpose(1, 2);print(x.shape)
        x = torch.reshape(x, [-1, self.embed_dim, *self.final_shape])
        return x

    def forward(self, x):
        ### we assume always feed the tensor (B, p*z, h, w)
        B, P ,h, w = x.shape
        x = x.reshape(B,-1,*self.img_size)
        #timer.restart(level=0)
        x = self.forward_features(x)
        #timer.record('forward_features',level=0)
        x = self.final_dropout(x)
        #timer.record('final_dropout',level=0)
        x = self.pre_logits(x)
        #timer.record('pre_logits',level=0)
        x = self.head(x)
        #timer.record('head',level=0)
        x = x.reshape(B,P,h,w)
        #timer.show_stat()
        #print("============================")

        return x

In [ ]:
model = AFNONet(img_size=[10,32,64],patch_size=(1,8,8),depth=1)

In [ ]:
a = torch.randn(1,20,10,32,64)

In [ ]:
a = torch.randn(1,3,4,32,64).flatten(1,2)
torch.dist(a.reshape(1,-1,4,32,64).reshape(1,12,32,64),a)

In [ ]:
a = torch.randn(1,12,32,64)
torch.dist(a.reshape(1,3,4,32,64).reshape(1,-1,32,64),a)

##### EulerEquationModel2

In [ ]:
class EulerEquationModel2(nn.Module):
    def __init__(self, args, backbone):
        super().__init__()
        self.Dx= First_Derivative_Layer(position=-1, dim=3, mode=2)
        self.Dy= First_Derivative_Layer(position=-2, dim=3, mode=2)
        self.Dz= First_Derivative_Layer(position=-3, dim=3, mode=1)
        self.thermal_factor  = nn.Parameter(torch.randn(3).reshape(1,3,1,1))
        self.alpha = nn.Parameter(torch.randn(1))
        #self.p_list         = nn.Parameter(torch.Tensor([10,8.5,5]).reshape(1,3,1,1),requires_grad=False)
        self.backbone =  backbone
        self.monitor = True
    def forward(self, Field):
        # u^{t+1} &= u^{t} + F_x - \nabla (Vu)  + u \nabla\cdot V - \partial_x\phi\\
        # v^{t+1} &= v^{t} + F_y - \nabla (Vv)  + v \nabla\cdot V - \partial_y\phi\\
        # T^{t+1} &= T^{t} + Q/C_v + \frac{RT}{C_pp}\omega  - \nabla (VT) +T \nabla\cdot V\\
        # \phi^{t+1}&=\phi^{t} + wg  - \nabla (V\phi)+ \phi \nabla\cdot V \\
        # 0&\approx \nabla\cdot V
        # input -> Field  = [u ,v, T, p] --> (Batch, 4, z, y ,x)
        # need generate unknown data [Fx, Fy , Q, W, o]
        b, si_z, i_y, i_x = Field.shape
        s=4
        i_z= si_z//4
        MachineLearningPart = self.backbone(Field).reshape(b, s+1, i_z, i_y, i_x) #(Batch, 5, z, y ,x)
        ExternalForce = MachineLearningPart[:,:4] #(Batch, 4, z, y ,x)
        o = MachineLearningPart[:,4:5] #(Batch, 1, z, y ,x)
        Field = Field.reshape(b, s, i_z, i_y,  i_x) #(Batch, 5, z, y ,x)
        u = Field[:,0:1]#(Batch, 1, z, y ,x)
        v = Field[:,1:2]#(Batch, 1, z, y ,x)
        T = Field[:,2:3]#(Batch, 1, z, y ,x)
        p = Field[:,3:4]#(Batch, 1, z, y ,x)
        V = torch.cat([u,v,o],1)#(Batch, 3, z, y ,x)
        Nabla_cdot_V = (self.Dx(u[:,0]) + self.Dy(v[:,0]) + self.Dz(o[:,0])).unsqueeze(1)#(Batch, 1, z, y ,x)
        Nabla_V_Field= Nabla_cdot_V*Field #(Batch, 4, z, y ,x)
        Vphysics     = torch.stack([V*u,V*v,V*T,V*p],1)#(Batch,4, 3, z, y ,x)
        Vphysics_dx = self.Dx(Vphysics[:,:,0].flatten(0,1)).reshape(Field.shape)#(Batch, 4, z, y ,x)
        Vphysics_dy = self.Dy(Vphysics[:,:,1].flatten(0,1)).reshape(Field.shape)#(Batch, 4, z, y ,x)
        Vphysics_dz = self.Dz(Vphysics[:,:,2].flatten(0,1)).reshape(Field.shape)#(Batch, 4, z, y ,x)
        PhysicsPart = -Vphysics_dx - Vphysics_dy - Vphysics_dz + Nabla_V_Field #(Batch,4,z, y ,x)
        InteractionPart= torch.stack([self.Dx(p[:,0]),
                                      self.Dy(p[:,0]),
                                      self.thermal_factor*T[:,0]*o[:,0]],1)#(Batch,3,z, y ,x)
        InteractionPart= F.pad(InteractionPart,(0,0,0,0,0,0,0,1)) #(Batch,4,z, y ,x)
        Delta_Fd     = ExternalForce + self.alpha*(PhysicsPart + InteractionPart)
        Field        = Field+ Delta_Fd
        constrain    = Nabla_V_Field
        if not self.monitor:
            return Field.flatten(1,2),(constrain**2).mean()
        else:
            return Field.flatten(1,2),(constrain**2).mean(),{"ExternalForceFactor":(ExternalForce**2).mean().item(),
                                                             "PhysicsDrivenFactor":(PhysicsPart**2).mean().item(),
                                                             "InteractionPart":(InteractionPart**2).mean().item(),
                                                             "alpha":self.alpha.item()}
